#Lab3-Part1: Sequential

In [ ]:
%%writefile sequential_kernel.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

//Method to read input files 1,2,4
int read_input_one_two_four(int **input1, char* filepath){
 FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));

    int temp1;

    while (fscanf(fp, "%d", &temp1) == 1) {
        (*input1)[counter] = temp1;

        counter++;
    }

    fclose(fp);
    return len;
}

//Method to read input file 3
int read_input_three(int** input1, int** input2, int** input3, int** input4,char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));
    *input2 = ( int *)malloc(len * sizeof(int));
    *input3 = ( int *)malloc(len * sizeof(int));
    *input4 = ( int *)malloc(len * sizeof(int));



    int temp1;
    int temp2;
    int temp3;
    int temp4;
    while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4) {
        (*input1)[counter] = temp1;
        (*input2)[counter] = temp2;
        (*input3)[counter] = temp3;
        (*input4)[counter] = temp4;
        counter++;
    }

    fclose(fp);
    return len;  
}

//Method to evaluate the inputs to the gate. The input consists of an input and the output of a previous node, and the gate type
/*
*gateType = 0 is AND, 1 is OR, 2 is NAND, 3 is NOR, 4 is XOR, and 5 is XNOR
*/
int gateEval(int gateType, int prevNodeOutput, int nodeInput){
    //AND Gate
    if(gateType == 0){
        if(nodeInput == 1 && prevNodeOutput == 1)return 1;
        return 0;
    }

    //OR Gate
    else if(gateType == 1){
        if(nodeInput == 1 || prevNodeOutput == 1)return 1;
        return 0;
    }

    //NAND Gate
    else if(gateType == 2){
        if(nodeInput == 1 && prevNodeOutput == 1)return 0;
        return 1;
    }

    //NOR Gate
    else if(gateType == 3){
        if(nodeInput == 0 && prevNodeOutput == 0)return 1;
        return 0;
    }

    //XOR Gate
    else if(gateType == 4){
        if(nodeInput == prevNodeOutput)return 0;
        return 1;
    }

    //XNOR Gate
    else if(gateType == 5){
        if(nodeInput == prevNodeOutput)return 1;
        return 0;
    }
    else{
        return -1;
    }
   
}

int main(int argc, char *argv[]) {
  if(argc != 7){
      printf("ERROR: Input format was incorrect.\n");
  }  



  // Variables
  int numNodePtrs;
  int numNodes;
  int *nodePtrs_h;
  int *nodeNeighbors_h;
  int *nodeVisited_h;
  int numTotalNeighbors_h;
  int *currLevelNodes_h;
  int numCurrLevelNodes;
  int numNextLevelNodes_h = 0;
  int *nodeGate_h;
  int *nodeInput_h;
  int *nodeOutput_h;


  numNodePtrs = read_input_one_two_four(&nodePtrs_h, argv[1]);
  numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, argv[2]);
  numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, argv[3]);
  numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, argv[4]);

  //output
  int *nextLevelNodes_h = (int*)malloc(numNodes * sizeof(int));

  //Begin timer -- we will only time the execution time of the main logic
  clock_t begin = clock();

  //Loop over all nodes at the current level
  for(int i = 0; i < numCurrLevelNodes; i++){
      int node = currLevelNodes_h[i];

      //Loop over all neighbors of the node
      for(int j = nodePtrs_h[node]; j < nodePtrs_h[node+1]; j++){
          int neighbor = nodeNeighbors_h[j];

          if(nodeVisited_h[neighbor] == 0){
              nodeVisited_h[neighbor] = 1;
              nodeOutput_h[neighbor] = gateEval(nodeGate_h[neighbor], nodeOutput_h[node], nodeInput_h[neighbor]);
              nextLevelNodes_h[numNextLevelNodes_h] = neighbor;
              numNextLevelNodes_h++;
          }
      }
  }

  //End timer
  clock_t end = clock();
  double time_spent = (double)(end - begin)/CLOCKS_PER_SEC;

  printf("Time Spent: %f seconds.\n", time_spent);

  //Finished evaluation, write the output
  //One output file contains the length of nodeOutput and all its elements
  //Second output file contains the length of numNextLevelNodes and all of its elements

  FILE *nodeOutputFile = fopen(argv[5], "w");
  FILE *nextLevelNodesOutputFile = fopen(argv[6], "w");

  if(!nodeOutputFile || !nextLevelNodesOutputFile){
      printf("ERROR: Could not open one of the output files.");
      return -1;
  }

  //Do the nodeOutputFile first
  fprintf(nodeOutputFile, "%d\n", numNodes);
  for(int i = 0; i < numNodes; i++){
      fprintf(nodeOutputFile, "%d\n", nodeOutput_h[i]);
  }

  //Do the nextLevelNodes output file next
  fprintf(nextLevelNodesOutputFile, "%d\n", numNextLevelNodes_h);
  for(int i = 0; i < numNextLevelNodes_h; i++){
      fprintf(nextLevelNodesOutputFile, "%d\n", nextLevelNodes_h[i]);
  }

  fclose(nodeOutputFile);
  fclose(nextLevelNodesOutputFile);

  return 0;
}


Overwriting sequential_kernel.cu


In [ ]:
!nvcc sequential_kernel.cu -o sequential

sequential_kernel.cu(119): warning: variable "numNodePtrs" was set but never used

sequential_kernel.cu(124): warning: variable "numTotalNeighbors_h" was set but never used



In [ ]:
!./sequential input1.raw input2.raw input3.raw input4.raw nodeOutputFile.raw nextLevelNodesFile.raw

Time Spent: 0.002425 seconds.


#Lab3-Part2: Global Queuing

In [ ]:
%%writefile parallel_kernel.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

//Method to evaluate the inputs to the gate. The input consists of an input and the output of a previous node, and the gate type
/*
*gateType = 0 is AND, 1 is OR, 2 is NAND, 3 is NOR, 4 is XOR, and 5 is XNOR
*/
__device__ int gateEval(int gateType, int prevNodeOutput, int nodeInput){
    //AND Gate
    if(gateType == 0){
        if(nodeInput == 1 && prevNodeOutput == 1)return 1;
        return 0;
    }
    //OR Gate
    else if(gateType == 1){
        if(nodeInput == 1 || prevNodeOutput == 1)return 1;
        return 0;
    }
    //NAND Gate
    else if(gateType == 2){
        if(nodeInput == 1 && prevNodeOutput == 1)return 0;
        return 1;
    }
    //NOR Gate
    else if(gateType == 3){
        if(nodeInput == 0 && prevNodeOutput == 0)return 1;
        return 0;
    }
    //XOR Gate
    else if(gateType == 4){
        if(nodeInput == prevNodeOutput)return 0;
        return 1;
    }
    //XNOR Gate
    else if(gateType == 5){
        if(nodeInput == prevNodeOutput)return 1;
        return 0;
    }
    else{
        return -1;
    } 
}

//Parallel Code
__global__ void global_queuing_kernel(int* nextLevelNodes_gpu, int* nodeVisited_gpu, int* nodeOutput_gpu, int* numNextLevelNodes_gpu, int* nodeInput_h, int numCurrLevelNodes, int* currLevelNodes_h, int* nodePtrs_h, int* nodeNeighbors_h, int* nodeGate_h, int numBlocks, int numThreads){
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    while(index < numCurrLevelNodes){
        int node = currLevelNodes_h[index];

        //Loop over neighbors of the node
        for(int i = nodePtrs_h[node]; i < nodePtrs_h[node+1]; i++){
            int neighbor = nodeNeighbors_h[i];

            //Perform an atomic compare-and-swap. If the value at nodeVisited is 0 we replace it by 1 and return 0
            //If the value at nodeVisited is 1, we do NOT swap, and return 1
            if(atomicCAS(&nodeVisited_gpu[neighbor],0,1) == 0){
                //Calculate the gate output
                nodeOutput_gpu[neighbor] = gateEval(nodeGate_h[neighbor], nodeOutput_gpu[node], nodeInput_h[neighbor]);
                
                //This line is important because the value of numNextLevelNodes_gpu is shared by all blocks/threads
                //So we perform an atomic exchange to assign the value and increment the count atomically
                atomicExch(&nextLevelNodes_gpu[atomicAdd(&numNextLevelNodes_gpu[0], 1)], neighbor);
            }
        }
        //Index increases by the total number of threads
        index += (numBlocks*numThreads);
    }
}


//Method to read input files 1,2,4
int read_input_one_two_four(int **input1, char* filepath){
 FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));

    int temp1;

    while (fscanf(fp, "%d", &temp1) == 1) {
        (*input1)[counter] = temp1;

        counter++;
    }

    fclose(fp);
    return len;
}

//Method to read input file 3
int read_input_three(int** input1, int** input2, int** input3, int** input4,char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));
    *input2 = ( int *)malloc(len * sizeof(int));
    *input3 = ( int *)malloc(len * sizeof(int));
    *input4 = ( int *)malloc(len * sizeof(int));



    int temp1;
    int temp2;
    int temp3;
    int temp4;
    while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4) {
        (*input1)[counter] = temp1;
        (*input2)[counter] = temp2;
        (*input3)[counter] = temp3;
        (*input4)[counter] = temp4;
        counter++;
    }

    fclose(fp);
    return len;  
}

int main(int argc, char *argv[]) {
  if(argc != 7){
      printf("ERROR: Input format was incorrect.\n");
  }  

  // Variables
  int numNodePtrs;
  int numNodes;
  int *nodePtrs_h;
  int *nodeNeighbors_h;
  int *nodeVisited_h;
  int numTotalNeighbors_h;
  int *currLevelNodes_h;
  int numCurrLevelNodes;
  int *nodeGate_h;
  int *nodeInput_h;
  int *nodeOutput_h;


  numNodePtrs = read_input_one_two_four(&nodePtrs_h, argv[1]);
  numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, argv[2]);
  numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, argv[3]);
  numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, argv[4]);

  //output and gpu variables
  int *nextLevelNodes_gpu;
  int *nodeVisited_gpu;
  int *nodeOutput_gpu;
  int *numNextLevelNodes_gpu;
  int *nodeInput_gpu;
  int *currLevelNodes_gpu;
  int *nodePtrs_gpu;
  int *nodeNeighbors_gpu;
  int *nodeGate_gpu;

  //Alocate gpu memory here
  cudaMallocManaged((void**)&nextLevelNodes_gpu, (numNodes * sizeof(int)));
  cudaMallocManaged((void**)&nodeVisited_gpu, (numNodes * sizeof(int)));
  cudaMallocManaged((void**)&nodeOutput_gpu, (numNodes * sizeof(int)));
  cudaMallocManaged((void**)&numNextLevelNodes_gpu, sizeof(int));
  cudaMallocManaged((void**)&nodeInput_gpu, (numNodes * sizeof(int)));
  cudaMallocManaged((void**)&currLevelNodes_gpu, (numCurrLevelNodes * sizeof(int)));
  cudaMallocManaged((void**)&nodePtrs_gpu, (numNodePtrs * sizeof(int)));
  cudaMallocManaged((void**)&nodeNeighbors_gpu, (numTotalNeighbors_h * sizeof(int)));
  cudaMallocManaged((void**)&nodeGate_gpu, (numNodes * sizeof(int)));

  //Populate the gpu mem where necessary
  numNextLevelNodes_gpu[0] = 0;

  for(int i = 0; i < numNodes; i++){
      nodeVisited_gpu[i] = nodeVisited_h[i];
      nodeOutput_gpu[i] = nodeOutput_h[i];
      nodeInput_gpu[i] = nodeInput_h[i];
      nodeGate_gpu[i] = nodeGate_h[i];
  }

  for(int i = 0; i < numCurrLevelNodes; i++){
      currLevelNodes_gpu[i] = currLevelNodes_h[i];
  }

  for(int i = 0; i < numTotalNeighbors_h; i++){
      nodeNeighbors_gpu[i] = nodeNeighbors_h[i];
  }

  for(int i = 0; i < numNodePtrs; i++){
      nodePtrs_gpu[i] = nodePtrs_h[i];
  }

  //Kernel Call
  int numBlocks = 10;
  int numThreads = 32;

  //Start timer here
  clock_t begin = clock();
  
  global_queuing_kernel<<<numBlocks, numThreads>>>(nextLevelNodes_gpu, nodeVisited_gpu, nodeOutput_gpu, numNextLevelNodes_gpu, nodeInput_gpu, numCurrLevelNodes, currLevelNodes_gpu, nodePtrs_gpu, nodeNeighbors_gpu, nodeGate_gpu, numBlocks, numThreads);
  cudaDeviceSynchronize();

  //Finish clock here
  clock_t end = clock();
  double time_spent = (double)(end - begin)/CLOCKS_PER_SEC;

  printf("Time Spent: %f seconds. Number of Threads: %d. Number of Blocks: %d.\n", time_spent, numThreads, numBlocks);

  //Finished evaluation, write the output
  //One output file contains the length of nodeOutput and all its elements
  //Second output file contains the length of numNextLevelNodes and all of its elements

  FILE *nodeOutputFile = fopen(argv[5], "w");
  FILE *nextLevelNodesOutputFile = fopen(argv[6], "w");

  if(!nodeOutputFile || !nextLevelNodesOutputFile){
      printf("ERROR: Could not open one of the output files.");
      return -1;
  }

  //Do the nodeOutputFile first
  fprintf(nodeOutputFile, "%d\n", numNodes);
  for(int i = 0; i < numNodes; i++){
      fprintf(nodeOutputFile, "%d\n", nodeOutput_gpu[i]);
  }

  //Do the nextLevelNodes output file next
  fprintf(nextLevelNodesOutputFile, "%d\n", numNextLevelNodes_gpu[0]);
  for(int i = 0; i < numNextLevelNodes_gpu[0]; i++){
      fprintf(nextLevelNodesOutputFile, "%d\n", nextLevelNodes_gpu[i]);
  }

  //Close files
  fclose(nodeOutputFile);
  fclose(nextLevelNodesOutputFile);

  //free all memory
  cudaFree(nextLevelNodes_gpu);
  cudaFree(nodeVisited_gpu);
  cudaFree(nodeOutput_gpu);
  cudaFree(numNextLevelNodes_gpu);

  return 0;
}


Overwriting parallel_kernel.cu


In [ ]:
!nvcc parallel_kernel.cu -o global_queuing
!./global_queuing input1.raw input2.raw input3.raw input4.raw nodeOutputFile.raw nextLevelNodesFile.raw

Time Spent: 0.002026 seconds. Number of Threads: 32. Number of Blocks: 10.


#Tester Code

##Compare Next Level Nodes

In [ ]:
%%writefile compareNextLevelNodes.cu


#include <stdio.h>
#include <stdlib.h>
void sort(int *pointer, int size){
  //get from https://stackoverflow.com/questions/13012594/sorting-with-pointers-instead-of-indexes
    int *i, *j, temp;
    for(i = pointer; i < pointer + size; i++){
        for(j = i + 1; j < pointer + size; j++){
            if(*j < *i){
                temp = *j;
                *j = *i;
                *i = temp;
            }
        }
    }
}

void compareNextLevelNodeFiles(char *file_name1, char *file_name2) 
{ 

  
    FILE* fp_1 = fopen(file_name1, "r");
    if (fp_1 == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 

    FILE* fp_2 = fopen(file_name2, "r");
    if (fp_2 == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len_1;
    int len_2;
    int length_file_1 = fscanf(fp_1, "%d", &len_1);
    int length_file_2 = fscanf(fp_2, "%d", &len_2);
    printf("%d, %d\n", len_1, len_2);
    if(length_file_1 != length_file_2){
      fprintf(stderr, "Wrong file length\n");
      exit(1);
    }
    int *input1 = (int *)malloc(len_1 * sizeof(int));
    int *input2 = (int *)malloc(len_2 * sizeof(int));




    int temp1;
    int temp2;

    while ((fscanf(fp_1, "%d", &temp1) == 1) && (fscanf(fp_2, "%d", &temp2) == 1)) {
        (input1)[counter] = temp1;
        (input2)[counter] = temp2;
        counter++;
    }

    sort(input1, len_1);
    sort(input2, len_2);

    for(int i=0; i< len_1; i++){
      if(input1[i] != input2[i]){
        fprintf(stderr, "Something goes wrong\n");
        exit(1);
      }
    }

    fprintf(stderr, "No errors!\n");
        exit(1);

} 

int main(int argc, char *argv[]){

    if( argc < 3) {
      printf("Require two files\n");
      exit(1);
      
   }
compareNextLevelNodeFiles(argv[1], argv[2]);
}


Writing compareNextLevelNodes.cu


##Compare Node Output

In [ ]:
%%writefile compareNodeOutput.cu

#include <stdio.h>
#include <stdlib.h>


void compareFiles(char *file_name1, char *file_name2) 
{ 
//get from https://www.tutorialspoint.com/c-program-to-compare-two-files-and-report-mismatches
FILE* fp1 = fopen(file_name1, "r");
FILE* fp2 = fopen(file_name2, "r");
    // fetching character of two file 
    // in two variable ch1 and ch2 
    char ch1 = getc(fp1); 
    char ch2 = getc(fp2); 
  
    // error keeps track of number of errors 
    // pos keeps track of position of errors 
    // line keeps track of error line 
    int error = 0, pos = 0, line = 1; 
  
    // iterate loop till end of file 
    while (ch1 != EOF && ch2 != EOF) 
    { 
        pos++; 
  
        // if both variable encounters new 
        // line then line variable is incremented 
        // and pos variable is set to 0 
        if (ch1 == '\n' && ch2 == '\n') 
        { 
            line++; 
            pos = 0; 
        } 
  
        // if fetched data is not equal then 
        // error is incremented 
        if (ch1 != ch2) 
        { 
            error++; 
            printf("Line Number : %d \tError"
               " Position : %d \n", line, pos); 
        } 
  
        // fetching character until end of file 
        ch1 = getc(fp1); 
        ch2 = getc(fp2); 
    } 
  
    printf("Total Errors : %d\t", error); 
} 

int main(int argc, char *argv[]){

    if( argc < 3) {
      printf("Require two files\n");
      exit(1);
      
   }
compareFiles(argv[1], argv[2]);
}


Writing compareNodeOutput.cu


##Evaluate

In [ ]:
!nvcc compareNextLevelNodes.cu -o compareNextLevelNodes
!nvcc compareNodeOutput.cu -o compareNodeOutput

In [ ]:
!./compareNextLevelNodes sol_nextLevelNodes.raw nextLevelNodesFile.raw

40101, 40101
No errors!


In [ ]:
!./compareNodeOutput sol_nodeOutput.raw nodeOutputFile.raw

Total Errors : 0	